# Community Enrichment Analysis

Here I perform Gene Ontology enrichment analysis on each of the communities produced by the infomap community detection algorithm, detailed in the following notebook:

https://github.com/wigasper/viral-mirna-database/blobl/master/exploratory_network_analysis.md

I am interested in the what proportion of community members share any significantly enriched Gene Ontology terms. This may provide insight as to why target genes or communities of target genes ended up as targets of Epstein-Barr virus, and it also may potentially serve as validation for the infomap community detection algorithm. Some of this is fairly intuitive and to be expected: gene products that interact with each other tend to be part of the same biological processes and located in the same locations. Thus, it could be expected that each community will have significantly enriched GO terms, and many members of each community will share some of these significantly enriched GO terms.

I used the [GOATools package](https://github.com/tanghaibao/goatools) with their [Run a Gene Ontology Enrichment Analysis notebook](https://github.com/tanghaibao/goatools/blob/master/notebooks/goea_nbt3102.ipynb) as a guide to help me get started and provide some code.

In [ ]:
import psycopg2
from goatools.obo_parser import GODag
from goatools.associations import read_ncbi_gene2go
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT as gene_id_2_nt
from goatools.go_enrichment import GOEnrichmentStudy
from tqdm import tqdm_notebook

## Conversion of Gene Symbols to Gene IDs
I conducted my [exploratory analysis](https://github.com/wigasper/viral-mirna-database/blobl/master/exploratory_network_analysis.md) using gene symbols, and I need to convert these to gene IDs to play nicely with GOATools. To do this I use psycopg2 to query my database (which contains both symbols and gene IDs for each gene) and make a dictionary to do the conversion.

In [4]:
# Create connection
conn = psycopg2.connect(database="vir_mirna", user="wkg", 
                        password="apples", host="localhost")

# Create cursor
cursor = conn.cursor()

# Query
cursor.execute("SELECT protein.symbol, protein.gene_id "
               "FROM protein, viral_mirna, viral_target "
               "WHERE protein.uniprot_id = viral_target.uniprot_id "
               "AND viral_target.vi_mirna_id = viral_mirna.vi_mirna_id "
               "AND viral_mirna.virus = 'EBV';")

results = cursor.fetchall()

# Make dict
gene_to_symbol = dict(results)

## Read in Community Data

In [5]:
# Get EBV targets to subset membership
ebv_targets = []
with open("./data/ebv_target_symbols.csv", "r") as fp:
    for line in fp:
        line = line.strip("\n")
        ebv_targets.append(line)

# Read in membership data and convert to symbols to gene IDs
membership = []
with open("./data/protein_community_membership.csv", "r") as fp:
    for line in fp:
        line = line.strip("\n")
        line = line.split(",")
        if line[0] in ebv_targets and gene_to_symbol[line[0]] is not None:
            membership.append([gene_to_symbol[line[0]], int(line[1])])

# Extract community IDs
communities = [comm[1] for comm in membership]
# Remove duplicates
communities = list(dict.fromkeys(communities))

## GO Enrichment Analysis Setup

In [6]:
obodag = GODag("./data/go.obo")
geneid2gos = read_ncbi_gene2go("./data/gene2go", taxids=[9606])

goeaobj = GOEnrichmentStudy(
        gene_id_2_nt.keys(), # human protein coding genes
        geneid2gos, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method

./data/go.obo: fmt(None) rel(None) 47,381 GO Terms
  20,404 items READ: ./data/gene2go
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 88% 18,480 of 20,913 population items found in association


In [ ]:
comm_enrich_scores = []
for community in tqdm_notebook(communities):
    subset = [gene[0] for gene in membership if gene[1] == community]

    goea_results = goeaobj.run_study(subset);
    sig_results = [result for result in goea_results if result.p_fdr_bh < 0.05]
    
    enriched_mem_ratios = []
    for res in sig_results:
        enriched_mem_ratios.append(res.ratio_in_study[0] / res.ratio_in_study[1])
    
    if len(enriched_mem_ratios):
        avg_enriched_ratio = sum(enriched_mem_ratios) / len(enriched_mem_ratios)
    else:
        avg_enriched_ratio = 0
    
    comm_enrich_scores.append([community, avg_enriched_ratio])

100%      3 of      3 study items found in association
100%      3 of      3 study items found in population(20913)
Calculating 17,999 uncorrected p-values using fisher_scipy_stats
  17,999 GO terms are associated with 18,480 of 20,913 population items
      39 GO terms are associated with      3 of      3 study items
       0 GO terms found significant (< 0.05=alpha) after multitest correction: statsmodels fdr_bh
100%      8 of      8 study items found in association
100%      8 of      8 study items found in population(20913)
Calculating 17,999 uncorrected p-values using fisher_scipy_stats
  17,999 GO terms are associated with 18,480 of 20,913 population items
     109 GO terms are associated with      8 of      8 study items
       8 GO terms found significant (< 0.05=alpha) after multitest correction: statsmodels fdr_bh
100%     11 of     11 study items found in association
100%     11 of     11 study items found in population(20913)
Calculating 17,999 uncorrected p-values using fi